In [1]:
import cv2
import numpy as np

In [8]:
def cropped_black_borders(img:np.ndarray, thr= 12, pad=0):
    # thr parametresi thresh değerini temsil eder. Arka planı siyah olan
    # görüntülerdeki nesneyi ayırt etmek (crop yapmak) için siyah rengine
    # yakın bir değer verdik

    # pad parametresi de padding'e karşılık gelir. Arka plandan, görüntüyü
    # seçtikten ve kırptıktan sonra etrafına (sağ, sol, üst, alt) ne kadar
    # boşluk bırakılacağını belirler

    # Eğer kullanıcının bize verdiği görüntü 3 kanallıysa, bunu gri seviyeye
    # dönüştürüyoruz
    
    if img.ndim == 3:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    # Görüntüdeki siyah olmayan pikselleri seçiyoruz
    mask = (gray>thr).astype(np.uint8)*255
    kernel = np.ones((5,5),dtype=np.uint8)

    thresh = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # kenar tespiti için eşikleme işleminden sonra kontur bulma işlemine geçiyoruz

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # seçilen kontur listesindenki en büyük konturu seçiyoruz
    c = max(contours, key=cv2.contourArea)

    # Seçilen konturu da bir dikdörtgen haline getiriyoruz
    x,y,w,h = cv2.boundingRect(c)
    
    # Kontora göre seçilen elemanın (arka plandan ayırt edilmek istenen
    # değişkenin) orijinal görüntü boyutlarından taşmaması için önlem alacağız

    # Önce yüksekliği ve genişliği, görüntünün kendisinden alıyoruz

    H, W = img.shape[:2]

    x1 = max(0, x - pad) # sol taraftan taşma olmamasını sağlar
    y1 = max(0, y - pad) # sağ taraftan taşma olmamasını sağlar
    x2 = min(W, x + w + pad) # yüksekliğin taşmamasını sağlar
    y2 = min(H, y + h + pad) # genişliğin taşmamasını sağlar

    return img[y1:y2, x1:x2]

In [9]:
# yukarıda, fonksiyonumuzu tanımladık, aşağıda istediğimiz görüntüyle
# çalıştıracağız

my_bgr_img = cv2.imread("./yeni1.png")

cropped_img = cropped_black_borders(my_bgr_img, 10, 0)

cv2.imshow("Cropped Image",cropped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()